# Clustering Crypto

In [55]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [56]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [57]:
# Loading data and create DataFrame with neccessary columns
file_path = Path("./Resources/crypto_data.csv")
crypto_data_df = pd.read_csv(file_path)
crypto_data_df = crypto_data_df.drop(columns=["Unnamed: 0"])
crypto_data_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


### Data Preprocessing

In [58]:
# Keep only cryptocurrencies that are trading
crypto_data_df = crypto_data_df.loc[crypto_data_df["IsTrading"] == True]
crypto_data_df.head(15)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [59]:
# Keep only cryptocurrencies with a working algorithm
crypto_data_df = crypto_data_df.loc[crypto_data_df["Algorithm"] != "N/A"]
crypto_data_df.head(15)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
6,2015 coin,X11,True,PoW/PoS,NaN,0
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,True,PoW,1.076842e+08,0
9,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [60]:
# Remove the "IsTrading" column
crypto_data_df = crypto_data_df.drop(columns=["IsTrading"])
crypto_data_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [61]:
# Remove rows with at least 1 null value
crypto_data_df = crypto_data_df.dropna()
crypto_data_df.head(15)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
11,Monero,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [62]:
# Remove rows with cryptocurrencies having no coins mined
crypto_data_df = crypto_data_df.loc[crypto_data_df['TotalCoinsMined'] != 0]
crypto_data_df.head(15)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
11,Monero,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
13,ZCash,Equihash,PoW,7.383056e+06,21000000


In [63]:
# Drop rows where there are 'N/A' text values
for column in crypto_data_df:
    crypto_data_df = crypto_data_df.loc[crypto_data_df[column] != "N/A"]
crypto_data_df.head(15)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
9,Litecoin,Scrypt,PoW,6.303924e+07,84000000
10,Dash,X11,PoW/PoS,9.031294e+06,22000000
11,Monero,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
13,ZCash,Equihash,PoW,7.383056e+06,21000000


In [64]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = crypto_data_df[["CoinName"]]
coin_name_df.head(15)

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum
9,Litecoin
10,Dash
11,Monero
12,Ethereum Classic
13,ZCash


In [65]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_data_df = crypto_data_df.drop(columns=["CoinName"])
crypto_data_df.head(15)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0
9,Scrypt,PoW,6.303924e+07,84000000
10,X11,PoW/PoS,9.031294e+06,22000000
11,CryptoNight-V7,PoW,1.720114e+07,0
12,Ethash,PoW,1.133597e+08,210000000
13,Equihash,PoW,7.383056e+06,21000000


In [66]:
# Create dummy variables for text features
text_feature_variable = ''

# Import Label Encoder from SKLearn and encode categorical data:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(crypto_data_df['Algorithm'])
crypto_data_df['Algorithm'] = le.transform(crypto_data_df['Algorithm'])

le.fit(crypto_data_df['ProofType'])
crypto_data_df['ProofType'] = le.transform(crypto_data_df['ProofType'])


crypto_data_df.head(15)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,53,16,4.199995e+01,42
2,53,16,1.055185e+09,532000000
5,67,16,2.927942e+10,314159265359
7,47,13,1.792718e+07,21000000
8,20,13,1.076842e+08,0
9,53,13,6.303924e+07,84000000
10,65,16,9.031294e+06,22000000
11,12,13,1.720114e+07,0
12,20,13,1.133597e+08,210000000
13,18,13,7.383056e+06,21000000


In [69]:
# Standardize data
scaler = StandardScaler()

crypto_scaled = scaler.fit_transform(crypto_data_df)
crypto_scaled

array([[ 0.39335561,  0.8871567 , -0.11674788, -0.15286468],
       [ 0.39335561,  0.8871567 , -0.09358885, -0.14499604],
       [ 1.21439604,  0.8871567 ,  0.52587231,  4.4937636 ],
       ...,
       [-2.12841143,  0.00878917, -0.09523411, -0.13215444],
       [-1.65924547,  0.00878917, -0.11658774, -0.15255408],
       [ 0.39335561, -1.45515672, -0.11674507, -0.15284989]])

### Reducing Dimensions Using PCA

In [70]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [95]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(crypto_scaled)

pcs_df = pd.DataFrame(data=crypto_pca, columns=["PC_1", "PC_2", "PC_3"], index=crypto_data_df.index)

pcs_df.head(15)

,PC_1,PC_2,PC_3
0,-0.412665,0.823150,0.357227
2,-0.391306,0.827873,0.358336
5,3.143152,2.186748,0.487273
7,-0.190301,-0.003829,-0.043688
8,-0.051013,-1.161387,1.020473
9,-0.219679,0.253752,-0.280044
10,-0.473703,1.337814,-0.115708
11,-0.011450,-1.504754,1.335799
12,-0.048798,-1.160925,1.020929
13,-0.042085,-1.247450,1.099354


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [96]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_data_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

C:\Users\jayha\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [97]:
# Initialize the K-Means model
model = KMeans(n_clusters=3, random_state=5)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = crypto_data_df.join(pcs_df)
clustered_df['CoinName'] = coin_name_df['CoinName']
clustered_df['Class'] = predictions
clustered_df.head(15)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC_1,PC_2,PC_3,CoinName,Class
0,53,16,4.199995e+01,42,-0.412665,0.823150,0.357227,42 Coin,0
2,53,16,1.055185e+09,532000000,-0.391306,0.827873,0.358336,404Coin,0
5,67,16,2.927942e+10,314159265359,3.143152,2.186748,0.487273,EliteCoin,0
7,47,13,1.792718e+07,21000000,-0.190301,-0.003829,-0.043688,Bitcoin,0
8,20,13,1.076842e+08,0,-0.051013,-1.161387,1.020473,Ethereum,1
9,53,13,6.303924e+07,84000000,-0.219679,0.253752,-0.280044,Litecoin,0
10,65,16,9.031294e+06,22000000,-0.473703,1.337814,-0.115708,Dash,0
11,12,13,1.720114e+07,0,-0.011450,-1.504754,1.335799,Monero,1
12,20,13,1.133597e+08,210000000,-0.048798,-1.160925,1.020929,Ethereum Classic,1
13,18,13,7.383056e+06,21000000,-0.042085,-1.247450,1.099354,ZCash,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [89]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC_1",
    y="PC_1",
    z="PC_3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
    
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [87]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [90]:
# Print the total number of tradable cryptocurrencies
len(clustered_df.CoinName)

533

#### Scatter Plot with Tradable Cryptocurrencies

In [91]:
# Scale data to create the scatter plot


In [93]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)